### Documentation

To read more about the search API, visit the [docs](https://www.elastic.co/docs/reference/elasticsearch/mapping-reference/dense-vector)

### Connect to ElasticSearch

In [2]:
from pprint import pprint
from elasticsearch import Elasticsearch

es = Elasticsearch('http://localhost:9200')
client_info = es.info()
print('Connected to Elasticsearch!')
pprint(client_info.body)

Connected to Elasticsearch!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'FQt-ffZfTpeh0Snf3pUAQw',
 'name': 'ae8b5b4be42b',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2024-08-05T10:05:34.233336849Z',
             'build_flavor': 'default',
             'build_hash': '1a77947f34deddb41af25e6f0ddb8e830159c179',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '9.11.1',
             'minimum_index_compatibility_version': '7.0.0',
             'minimum_wire_compatibility_version': '7.17.0',
             'number': '8.15.0'}}


### Inserting documents

Because we are using the `dense_vector` data type, we need to do the mapping manually.

In [3]:
es.indices.delete(index='my_index', ignore_unavailable=True)
es.indices.create(
    index="my_index",
    mappings={
        "properties":{
            "sides_length": {
                "type":"dense_vector",
                "dims":4
            },
            "shape" :{
                "type":"keyword"
            }
        }
    },
)
                

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

### 1. Valid Case

We specified that `sides_length` should be a vector with 4 values. Since this condition was met, the operation was executed successfully.

In [8]:
from pprint import pprint

response = es.index(
    index='my_index',
    id=1,
    document={
        "shape":"square",
        "sides_length":[5, 5, 5, 5],
    }
)

pprint(response.body)

{'_id': '1',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 2,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_version': 2,
 'result': 'updated'}


Let's check the mapping. We can see that `sides_length` is a `dense_vector` with 4 dimensions and `cosine` set as the similarity measure. These are parameters we can modify if needed. Since we didn’t specify them when manually setting the mapping, Elasticsearch automatically applied the default values.

In [9]:
pprint(es.indices.get_mapping(index='my_index').body)

{'my_index': {'mappings': {'properties': {'shape': {'type': 'keyword'},
                                          'side_length': {'type': 'long'},
                                          'sides_length': {'dims': 4,
                                                           'index': True,
                                                           'index_options': {'ef_construction': 100,
                                                                             'm': 16,
                                                                             'type': 'int8_hnsw'},
                                                           'similarity': 'cosine',
                                                           'type': 'dense_vector'}}}}}


### 2.Invalid Case

In this case, the operation fails because indexing a matrix is not supported with the `dense_vector` field type.

In [10]:
response = es.index(
    index= 'my_index',
    id=2,
    document={
        "shape":"square",
        "sides_length":[[5, 5], [5, 5]],
    }
)

pprint(response.body)

BadRequestError: BadRequestError(400, 'document_parsing_exception', 'Failed to parse object: expecting token of type [VALUE_NUMBER] but found [START_ARRAY]')